In [1]:
# Necessary imports of functions and dictionaries
# from dictionaries import *
# from sample_sheet_functions import *
import pandas as pd
from io import StringIO
from base64 import b64decode
import re


# Specific imports to clean up files
from dictionaries import different_24_dict, different_48_dict, different_96_dict, complete_sample_sheet_string, make_hiseq_header, make_miniseq_header, make_miseq_header, make_nextseq_header, f_primers_to_reverse_complement, flavours_list
from dash_sample_sheet_functions import *


###### For future
- Give possibility to download/upload in CSV/XLSX
- Table shows string on tooltip
- Table captures with a regular expression the sample name prefixes


# User download for template
_User Required to_
- Pick the table flavours they want (in form of dictionary key:True/False)
- Add project name, experiment name and additional comments (Now all stored in sample sheet template)

In [2]:
# Essential input
plates_options = {'24': False, '48': False, 'e': False, 'f': False, 'g': False, 'h': False, 'i': False, 'j': False, 'k': False, 'l': False}
project_name = ''

# Non-essential
experiment_name = ''
additional_comments = ''

file_format=None #Needs to be done at some point

In [3]:
# User input simulation
user_project_name = 'user project'
user_experiment_name = 'this is my project'
user_additional_comments = 'i want more space for the comments!'

user_plate_selection = {'e': False, 'f': True, 'g': False, 'h': True, 'i': False, 'j': False, 'k': True, 'l': True, 'tf': True, 'fe': False}


In [4]:
filename, sample_sheet = wrapper_make_template(user_plate_selection, user_project_name, user_experiment_name, user_additional_comments)

In [5]:
# with open('delete_now.csv', 'w') as file:
#     file.write(sample_sheet)


# User uploads template

_Persisting from before:_
- Project name
- Experiment name
- Comments 
- Flavours

_Requires user_ 
- Pick an instrument and the type of sample sheet required!

_In background_
- Sanity checks eg 
    - Checks number of samples
    - Checks number of blanks/controls
    - Checks no samples start with underscore (_)

_Processes:_
- Read file in, extract different tables with flavours and return it for it to display
- Pre process complete sample sheet?
- Get user selection and produce sample



In [6]:
#filename = 'delete_now.csv'
#filename = '96_well_template_4_plates.csv'
filename = 'template_Myproject.csv'


# Read file in 
with open(filename, 'r') as file:
    file_content = file.readlines()


In [7]:
#!!!!!!!!!!!!!!!!!!!!!!!!
tables, reports, metadata_dict, repeats = wrapper_handle_upload(file_content, filename)

# Check no two sample names are the same 

In [8]:
repeats # its the check

{}

In [9]:
reports

{'Plate_E': {'blanks': 0,
  'positive_controls': 0,
  'negative_controls': 0,
  'to_change': [],
  'total_samples': 96,
  'unique_sample_names': 96,
  'something_wrong': False},
 'Plate_I': {'blanks': 0,
  'positive_controls': 0,
  'negative_controls': 0,
  'to_change': [],
  'total_samples': 96,
  'unique_sample_names': 96,
  'something_wrong': False},
 'Plate_J': {'blanks': 0,
  'positive_controls': 0,
  'negative_controls': 0,
  'to_change': [],
  'total_samples': 96,
  'unique_sample_names': 96,
  'something_wrong': False},
 'Plate_24': {'blanks': 72,
  'positive_controls': 0,
  'negative_controls': 0,
  'to_change': [],
  'total_samples': 24,
  'unique_sample_names': 24,
  'something_wrong': False}}

In [10]:
# value = check_no_duplicates_or_unwanted_chars(tables)
# value

# Final step, prepare sample sheet based on requested machinery/format and submit!

_Required from user_
- Machinery selection
- Option to download

_Steps_:
- Get user input
- Get user's tables and metadata
- Melt everything in wrapper function that returns filename and file content


NEED TO ADD SOMETHING TO APPEND THE 24/48 well plates

In [11]:
# Data recouped from before 
metadata_dict = metadata_dict
tables = tables
project_name = metadata_dict['project_name']
experiment_name = metadata_dict['experiment_name']
comments = metadata_dict['comments']

In [12]:
possible_machinery_selection_forward_strand = ['mini_seq_standard_reagent', 'mi_seq', 'hi_seq_2500', 'hi_seq_2000','nova_seq_6000_v_1.5']

possible_machinery_selection_reverse_strand = ['iSeq', 'next_seq_system', 'hi_seq_x', 'hi_seq_4000', 'hi_seq_3000', 'mini_seq_rapid_reagent', 'nova_seq_6000_v_1.0']

In [13]:
# User selections
user_machinery_selection = 'mini_seq_standard'
user_sheet_complete = False


# User description file?



In [14]:
# Wrapper create complete sample sheet!!
def wrapper_create_complete_sample_sheet(tables, complete, machinery, project, experiment, comments):
    # Declare list of machineries based on required I5 index
    possible_machinery_selection_forward_strand = ['mini_seq_standard', 'mi_seq', 'hi_seq_2500', 'hi_seq_2000','nova_seq_6000_v_1.5']
    possible_machinery_selection_reverse_strand = ['iSeq', 'next_seq_system', 'hi_seq_x', 'hi_seq_4000', 'hi_seq_3000', 'mini_seq', 'nova_seq_6000_v_1.0']

    #Determine if the reverse complement is required!
    if machinery in possible_machinery_selection_forward_strand:
        reverse_complement = False
    elif machinery in possible_machinery_selection_reverse_strand:
        reverse_complement = True
    else:
        raise Exception('Machinery not recognised!')
    

    # Handle the presence of 24/48 plates
    tf_or_fe = None
    # Retain 24/48 well layouts, if any, and produce the sample sheet for the rest before
    if 'Plate_24' in tables.keys():
        tf_or_fe = ('Plate_24', tables['Plate_24']) 
        del tables['Plate_24']
    if 'Plate_48' in tables.keys():
        tf_or_fe = ('Plate_48', tables['Plate_48']) 
        del tables['Plate_48']
    
    # Make header from standard one
    header = make_generic_header(project, experiment, comments, machinery)

    # Get body of sample sheet
    sample_sheet_body = get_sample_sheet_body(tables, complete)

    # Make filename string starting from project name
    # If the filename has spaces in it, replace it with underscores
    project_name = project if ' ' not in project else project.replace(' ', '_')
    filename = f'{project_name}_sample_sheet{"_all_primers" if complete else ""}.csv'
    
    # IF the machinery requires REVERSE I5, substitute it in the sample sheet!
    # (map index names to correct sequence from the dictionary)
    if reverse_complement:
        sample_sheet_body['index2'] = sample_sheet_body['I5_Index_ID'].map(f_primers_to_reverse_complement)

    # Append 24/48 plate to it, if any
    if tf_or_fe:
        sample_sheet_body = add_tf_fe(sample_sheet_body, tf_or_fe, reverse_complement)

    # At this point in the old versions, you'd reduce the number of cols for mini_seq, given that we're standardising we won't (won't break the pipeline)
    
    #Prepare text to return
    text = header + sample_sheet_body.to_csv(index=False, header=False)

    return filename, text


In [15]:
filename,sample_sheet = wrapper_create_complete_sample_sheet(tables, user_sheet_complete, user_machinery_selection, user_project_name, user_experiment_name, user_additional_comments)

In [16]:
with open('sample_sheet.csv', 'w') as file:
    file.write(sample_sheet)

In [17]:
####### IT WORKS!!!! TRANSFER IT OVER (NEW SELF EXPLAINING NAMES)

In [18]:
# Get prefix

l = ['nomad', 'normal', 'nonstop', 'noob']

def common_prefix(lst):
    for s in zip(*lst):
        if len(set(s)) == 1:
            yield s[0]
        else:
            return

result = ''.join(common_prefix(l))
result

'no'

In [19]:
# LEFT TO DO
# ADD THE 24/48 (Complete function) (you have dictionaries to map already)
# Perform checks to see if everything is correct
# Implement in app